In [1]:
!pip install simpy pandas plotly

### Goal
- how to debug the process

In [ ]:
Last step we have seen that the monitoring show unexpected results. 

How can we feature out what is going on

In [2]:
from simpy import Environment, Process
import pandas as pd

In [3]:
class BasicProcessMonitor():  
    def __init__(self):
        self.data = []

    def trace(self, event): 
        print(f'Called back from {event} at {event.env.now:2.1f}')
        self.data.append({
            'name':event.name, 
            'startAt':event.startAt, 
            'stopAt':event.env.now,
            'duration':event.env.now - event.startAt + 1
        })
        
    def collect(self):
        return pd.DataFrame(self.data)

In [4]:
class MonitoredProcess(Process):  
    def __init__(self, env, action, name):
        super().__init__(env, action)
        # make sure process has a name and a start mark
        self.name = name
        self.startAt = env.now

class Visitor(MonitoredProcess):  
    def __init__(self, env, name):
        super().__init__(env, self.visit(), name)

    def visit(self): 
        print(f"{self.name}: Here I am at {self.env.now:2.1f}") 
        duration = len(self.name)
        yield env.timeout(duration)
        print(f"{self.name}: I must leave at {self.env.now:2.1f}") 

In [5]:
class InfiniteVisitorSource(Process):  
    def __init__(self, env):
        super().__init__(env, self.generate())
        self.count = 0
        self.monitor = BasicProcessMonitor()

    def generate(self): 
        while True:
            yield self.env.timeout(1)
            self.count += 1
            name = f"visitor-{self.count:03d}"
            print(f"start {name} at {self.env.now:2.1f}") 
            visitor = Visitor(self.env, name)
            visitor.callbacks.append(self.monitor.trace)

In [6]:
sim_duration = 15
env = Environment()
source = InfiniteVisitorSource(env)
print(f'start at {env.now}')
env.run(until=sim_duration)
print(f'stop at {env.now}')

start at 0
start visitor-001 at 1.0
visitor-001: Here I am at 1.0


NameError: name 'visitor' is not defined

In [ ]:
df_visitors = source.monitor.collect()
display(df_visitors)

In [ ]:
import plotly.express as px
fig = px.histogram(df_visitors, x="startAt", nbins=20)
fig.show()

In [ ]:
df_visitors[["duration"]].describe()

In [ ]:
import plotly.express as px
fig = px.histogram(df_visitors, x="duration", nbins=20)
fig.show()

TODO chronogram

More interedting visitor

hole -> not completedd

TODO chronogram -> clock

TODO logger -> show messageqs only when needed for debug

TODO fail / success + renege / multiple yield

In [ ]:
# new imports
import logging


In [ ]:
log_filename = "logs-5.log"
mainLogger = logging.getLogger()
mainLogger = logging.getLogger()
fhandler = logging.FileHandler(filename=log_filename, mode='w')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
mainLogger.addHandler(fhandler)
mainLogger.setLevel(logging.DEBUG)
mainLogger.debug("test")

# Logs

In [ ]:
with open(log_filename) as log:
            print(log.read())